In [2]:
import csv
import json
from time import sleep
from datetime import datetime
from random import random
from selenium.common import exceptions
from selenium import webdriver
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
def create_webdriver() :
    driver=webdriver.Chrome("C://Users//shaha//Downloads//chromedriver_win32//chromedriver.exe")
    return driver

In [5]:
def generate_url(search_term, page):
    base_template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    stem = base_template.format(search_term)
    url_template = stem + '&page={}'
    if page == 1:
        print(stem)
        return stem
    
    else:
        print(url_template.format(page))
        return url_template.format(page)


In [7]:
# Function to extract Product Title
def get_title(driver):
     
    try:
        # Outer Tag Object
        title = driver.find_element_by_xpath("//span[@id='productTitle']")
 
        # Inner NavigatableString Object
        title_value = title.text
 
        # Title as a string value
        title_string = title_value.strip()
 
 
    except Exception:
        title_string = "Unknown"   
 
    return title_string
 
# Function to extract Product Price
def get_price(soup):
 
    try:
        #xpathprice='//span[@id="price_inside_buybox"]'
#       price=driver.find_element_by_xpath(xpathprice)
#       print(price.text)
        #print("inside")
        price = soup.find_element_by_xpath("//span[@id='priceblock_ourprice']").text
        #print("Inside")
        #print(price)
 
    except Exception:
 
        try:
            # If there is some deal price
            price = soup.find_element_by_xpath("//span[@id='priceblock_saleprice']").text
 
        except:     
            price = ""  
 
    return price
 
# Function to extract Product Rating
def get_rating(soup):
 
    try:
        rating = soup.find_element_by_xpath("//span[@data-hook='rating-out-of-text']").text
        #print(rating)
         
    except Exception:
         
        try:
            rating = soup.find_element_by_xpath("//span[@data-hook='rating-out-of-text']").text
        except:
            rating = "" 
 
    return rating
 
# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find_element_by_xpath("//span[@id='acrCustomerReviewText']").text.strip()
         
    except Exception:
        review_count = ""   
 
    return review_count
 
# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find_element_by_xpath("//div[@id='availability']")
        available = available.find_element_by_tag_name("span").text.strip()
 
    except Exception:
        available = "Not Available"
 
    return available  


def get_product_img(soup):
    try:
        img=soup.find_element_by_xpath("//div[@id='imageBlock_feature_div']")
        img1=soup.find_element_by_xpath("//*[@id='landingImage']").get_attribute("src")
        #print(img1.get_attribute("src"))
    except:
        img1="No Image Link"
    return img1


def get_individual_rating_counts(driver):
    try:
        XPATH_AGGREGATE_RATING = '//table[@id="histogramTable"]//tr'
        total_ratings = driver.find_elements_by_xpath(XPATH_AGGREGATE_RATING)
        #print(total_ratings)
        ratings_dict={}
        ratings_list=[]
        for ratings in total_ratings:
            extracted_rating = ratings.find_elements_by_xpath('.//td//a')
            if extracted_rating:
                rating_key = extracted_rating[0].text
                raw_raing_value = extracted_rating[2]
                rating_value = raw_raing_value.text
                if rating_key:
                    ratings_dict.update({rating_key: rating_value})
                print(ratings_dict)
        ratings_list.append(ratings_dict)
                
    except:
        ratings_list="No Reviews"
    return ratings_list
        
def get_reviews_based_onfeatures(driver):
    try:
        feature_review=driver.find_elements_by_xpath(".//div[@data-hook='cr-summarization-attributes-list']")
        print(feature_review)
        feature=[]
        feature1=[]
        for product in feature_review:
            print("Inside for")
            ind=product.find_elements_by_xpath("//div[@class='a-fixed-right-grid-inner a-grid-vertical-align a-grid-center']//div[@class='a-fixed-right-grid-col a-col-left']//span")
            print(ind)

            ind3=product.find_elements_by_xpath("//div[@class='a-fixed-right-grid-inner a-grid-vertical-align a-grid-center']//div[@class='a-text-right a-fixed-right-grid-col a-col-right']//span[@class='a-size-base a-color-tertiary']")
            print(ind3)

            for i in ind:
                print(i.get_attribute("innerHTML"))
                feature.append(i.get_attribute("innerHTML"))
            for j in ind3:
                print("Here in :",j.get_attribute("innerHTML"))
                feature1.append(j.get_attribute("innerHTML"))


        res1 = {} 
        for v in feature: 
            for j in feature1: 
                    res1[v] = j 
                    feature1.remove(j) 
                    break  

        print ("Resultant dictionary is : " +  str(res1)) 

       
    except:
        res1="No feature based reviews"
    return res1 


def get_productdetails(soup):
    try:


        no_rows=len(driver.find_elements_by_xpath("//table[@class='a-normal a-spacing-micro']/tbody/tr"))
        print(no_rows)
        no_cols=len(driver.find_elements_by_xpath("//table[@class='a-normal a-spacing-micro']/tbody/tr[1]/td"))
        print(no_cols)
        
        data=[]
        for i in range(1,no_rows+1):
            ro=[]
            for j in range(1,no_cols+1):
                print(driver.find_element_by_xpath('//table[@class="a-normal a-spacing-micro"]/tbody/tr['+str(i)+']/td['+str(j)+']').text)
                ro.append(driver.find_element_by_xpath('//table[@class="a-normal a-spacing-micro"]/tbody/tr['+str(i)+']/td['+str(j)+']').text)
            data.append(ro)
        print(data)
 
        
    except:
        data=""
    
    return data

def get_other_details(soup):
    try:
        item_list=[]
        about_this_item=driver.find_elements_by_xpath('//*[@id="feature-bullets"]/ul/li')
        for item in about_this_item:
            item_list.append(item.text)
            print(item.text)
        product_description=driver.find_element_by_xpath('//*[@id="productDescription"]').text
        item_list.append(product_description)
        print(product_description.text)
    except:
            item_list="No description of the Product"

            
    return item_list
    

def get_relatedproducts(soup):
                count=0
                related_products=[]
                while count<=2:
                    print("Inside while")

                    #time.sleep(3)
                    try:

                        print("In here")
                        #WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable((By.XPATH, '//a[@class="a-button a-button-image a-carousel-button a-carousel-goto-nextpage"]')))
                        WebDriverWait(soup, 20).until(expected_conditions.visibility_of_element_located((By.XPATH, '//div[@class="a-carousel-col a-carousel-right"]')))
                        print("Next line")
                        nextpage=soup.find_element_by_xpath('//a[@class="a-button a-button-image a-carousel-button a-carousel-goto-nextpage"]').get_attribute('href')
                        soup.find_element_by_xpath('//a[@class="a-button a-button-image a-carousel-button a-carousel-goto-nextpage"]').click()
                        list=soup.find_elements_by_xpath('//div[@class="a-carousel-viewport"]')
                        print(list)
                       
                        for card in range(len(list)):
                            related=soup.find_element_by_xpath('//li[@class="a-carousel-card"]['+str(card+1)+']').text
                            related_products.append(related)
                            print(related)
                        count+=1

                    except Exception as e:
                        #break;
                        print("Exception",e)
                        count+=1
                        #condition=False
                        print("In except")
                return related_products

                print("After click()")
                
def get_review_comments(driver):
    try:
        review_comment_list=[]
        review_url=driver.find_element_by_xpath("//a[@data-hook='see-all-reviews-link-foot']").click()
        reviews_lists=driver.find_elements_by_xpath("//div[@data-hook='review']")
        print(reviews_lists)
        for review in reviews_lists:
            XPATH_RATING = './/i[@data-hook="review-star-rating"]//span[@class="a-icon-alt"]'
            raw_review_rating = review.find_element_by_xpath(XPATH_RATING).get_attribute("innerHTML")
            XPATH_REVIEW_HEADER = './/a[@data-hook="review-title"]//span'
            XPATH_REVIEW_POSTED_DATE = './/span[@data-hook="review-date"]'
            XPATH_AUTHOR = './/span[contains(@class,"profile-name")]'
            review_body=review.find_element_by_xpath(".//span[@data-hook='review-body']").text
            raw_review_header = review.find_element_by_xpath(XPATH_REVIEW_HEADER).text
            raw_review_posted_date = review.find_element_by_xpath(XPATH_REVIEW_POSTED_DATE).text
            raw_review_author = review.find_element_by_xpath(XPATH_AUTHOR).text
           
            review_dict={}
           
            review_dict = {
            
            'review_text': review_body,
            'review_posted_date': raw_review_posted_date,
            'review_header': raw_review_header,
            'review_rating': raw_review_rating,
            'review_author': raw_review_author
            }
            review_comment_list.append(review_dict)   
            print(review_comment_list)
    except:
        review_comment_list="No Reviews"
    return review_comment_list   


    
        
    
        

In [ ]:
import time
driver = create_webdriver()

for page in range(1, 2):  # max of 20 pages
            # load the next page
            search_url = generate_url('t-shirt', page)
            print(search_url)
            driver.get(search_url)
            driver.set_page_load_timeout(10)
            print('TIMEOUT while waiting for page to load')

           
            links=driver.find_elements_by_xpath('//a[@class="a-link-normal a-text-normal"]')
            link_list=[]
            for link in links:
                link_list.append(link.get_attribute("href"))
                #print(link_list)
            
            for link in link_list:
                driver.get(link)

                # Function calls to display all necessary product information
                print("Product Title =", get_title(driver))
                print("Product Price =", get_price(driver))
                print("Product Rating =", get_rating(driver))
                print("Number of Product Reviews =", get_review_count(driver))
                print("Availability =", get_availability(driver))
                print("Product Details =", get_productdetails(driver))
                print()
                print()
                
                print("Related Products:",get_relatedproducts(driver))
                
                title=get_title(driver)
                price=get_price(driver)
                rating=get_rating(driver)
                review_count=get_review_count(driver)
                availability=get_availability(driver)
                product_details=get_productdetails(driver)
                other_product_details=get_other_details(driver)
                related_product_details=get_relatedproducts(driver)
                individual_review_count=get_individual_rating_counts(driver)
                img_url=get_product_img(driver)
                reviews_basedonfeature=get_reviews_based_onfeatures(driver)
                print(reviews_basedonfeature)
                review_counts=get_review_comments(driver)
                
                
                final_data=[]
                data_dict={}
                data_dict["Title"]=title
                data_dict["Price"]=price
                data_dict["Rating"]=rating
                data_dict["Review_Count"]=review_count
                data_dict["Availability"]=availability
                data_dict["Details"]=product_details
                data_dict["Other Product Details"]=other_product_details
                data_dict["Related Product Details"]=related_product_details
                data_dict["Individual Rating Count"]=individual_review_count
                data_dict["Product IMG Link"]=img_url
                data_dict["Feature Based Review"]=reviews_basedonfeature
                data_dict["Review Comments"]=review_counts
                
                
                
                
                
                final_data.append(data_dict)
                f=open("output.json","a+")
                f.write(json.dumps(final_data,indent=2))
                f.close()
